In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout

In [5]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/train',
    labels='inferred',
    label_mode='int',
    batch_size=16,
    image_size=(128, 128)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/test',
    labels='inferred',
    label_mode='int',
    batch_size=16,
    image_size=(128, 128)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [6]:
# normalize
def process(image, label):
    image = tf.cast(image/255. ,tf.float32)
    return image, label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [7]:
# Feature extraction using a pre-trained CNN (VGG16)
from tensorflow.keras.applications import MobileNetV2
import numpy as np
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

def extract_features_fast(dataset):
    features = []
    labels = []
    for images, batch_labels in dataset.as_numpy_iterator():
        batch_features = base_model.predict(images, verbose=0)
        features.append(batch_features)
        labels.append(batch_labels)
    return np.vstack(features), np.concatenate(labels)

train_features, train_labels = extract_features_fast(train_ds)
validation_features, validation_labels = extract_features_fast(validation_ds)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:
# Reshape features for SVM
train_features = train_features.reshape(train_features.shape[0], -1)
validation_features = validation_features.reshape(validation_features.shape[0], -1)

In [9]:
# SVM Implementation
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(train_features, train_labels)

predictions = svm_classifier.predict(validation_features)
accuracy = accuracy_score(validation_labels, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9656


In [12]:
# prediction on new data
import cv2

# Load and preprocess the new image
test_img = cv2.imread('/content/dog.jpeg')
test_img = cv2.resize(test_img, (128, 128))
test_img = test_img.reshape((1, 128, 128, 3))

# Extract features from the new image using the pre-trained model
test_features = base_model.predict(test_img)
test_features = test_features.reshape(1, -1)  # Flatten features

# Make prediction using the SVM classifier
prediction = svm_classifier.predict(test_features)

# Interpret the prediction (0 for cat, 1 for dog)
if prediction[0] == 0:
    print("Predicted class: Cat")
else:
    print("Predicted class: Dog")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted class: Dog
